In [1]:
import sys
import os

# 현재 파일의 디렉토리 위치를 기준으로 경로 설정
current_dir = os.path.dirname(os.path.abspath('__file__')) 
nlp_utils_dir = os.path.abspath(os.path.join(current_dir, '..', 'NLP_utils'))

# NLP_utils 디렉토리를 sys.path에 추가
sys.path.append(nlp_utils_dir)

In [2]:
from selenium import webdriver
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import os
import time
import pandas as pd
import numpy as np
import re
from NLP_utils import *

In [3]:
def get_font_style(element):
    font_style = element.get_attribute("style")

    # font-style 속성을 추출하기 위한 정규 표현식 패턴
    font_style_pattern = r'font-style:\s*([^;]+)'

    # 정규 표현식을 이용하여 속성 추출
    match = re.search(font_style_pattern, font_style)

    if match:
        font_style = match.group(1)
    else:
        font_style = None
    return font_style

In [4]:
options = get_chrome_options()

In [5]:
options = get_chrome_options()
options.arguments.remove('--headless')

# 전체 뽑기

In [ ]:
from tqdm import tqdm

base_path = "http://www.davincimap.co.kr/davBase/Source/davSourList.jsp?job=C&tab=R&ClssID=DIRC000411&SelID=DIRC000413&worder=TITLE&work=TC"

s = time.time()

df = pd.DataFrame({"path": [],
                   "title": [],
                   "author": [],
                   "page_number": [],
                   "context": []})

driver = webdriver.Chrome(options=options)
driver.get(base_path)

WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "body")))      # 페이지 기다리기

#    headers = driver.find_elements(By.XPATH, r'/html/body/div[2]/div/div[2]/table[6]/tbody/tr')

#    new_headers = []
#    for i in range(len(headers)):
#        if i % 3 == 1:
#            new_headers.append(headers[i])
#    print(len(new_headers))

num = 137
for i in range(num):
    print(f"------------------------- {i+1}th / {num}th work start -------------------------")
    print()

    results = []
    
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "body")))

    header = driver.find_element(By.XPATH, f'/html/body/div[2]/div/div[2]/table[6]/tbody/tr[{3*i+2}]')
    
    tds = header.find_elements(By.CSS_SELECTOR, "td")

    meta_data = tds[2].find_element(By.CSS_SELECTOR, "div").find_element(By.CSS_SELECTOR, "a")

    path = meta_data.get_attribute("href")    # URL
    title = ""      # 제목
    author = ""     # 작가
    for m in meta_data.find_elements(By.CSS_SELECTOR, "span"):
        title += m.text
    try:
        author = tds[3].find_element(By.CSS_SELECTOR, "div").find_element(By.CSS_SELECTOR, "font").text      # 작가
    except NoSuchElementException as author_exception:
        author = tds[3].find_element(By.CSS_SELECTOR, "div").find_element(By.CSS_SELECTOR, "a").text      # 작가

    driver.get(path)

    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "body")))
    
    paragraphs = driver.find_elements(By.XPATH, r'/html/body/table[1]/tbody/tr/td[1]/table[3]/tbody/tr[1]/td/table/tbody/tr/td/div')

    page_number_first = 0
    is_seq = False
    body_text = ""  # 본문

    for parag in tqdm(paragraphs):
        p = parag.find_elements(By.CSS_SELECTOR, "div")
        
        try:
            if not is_seq:
                page_number_first = p[0].text     # page number
            pt = p[1].find_elements(By.CSS_SELECTOR, "span")
            font_style = get_font_style(pt[0])
            
            if not body_text:
                if font_style != "italic":
                    is_seq = False
                    for t in pt:
                        body_text += t.text
                    pair = [path, title, author, page_number_first, body_text]
                    results.append(pair)
                    body_text = ""
                else:
                    is_seq = True
                    for t in pt:
                        body_text += t.text + "\n"
            else:
                if font_style != "italic":
                    is_seq = False
                    pair = [path, title, author, page_number_first, body_text]
                    results.append(pair)

                    page_number_first = p[0].text
                    body_text = ""
                    for t in pt:
                        body_text += t.text
                    pair = [path, title, author, page_number_first, body_text]
                    results.append(pair)

                    body_text = ""
                else:
                    is_seq = True
                    for t in pt:
                        body_text += t.text        
        except IndexError as e1:
            pass
        except NoSuchElementException as e2:
            pass
    driver.back()   # 뒤로 가기
    e = time.time()

    df_add = pd.DataFrame(results,
                          columns=["path", "title", "author", "page_number", "context"])
    # 기존 데이터프레임과 병합

    df = pd.concat([df, df_add], axis=0, ignore_index=True)

    print(f"Total Elapsed time: {e-s:.2f}s")
    print()
    print(f"------------------------- Done {i+1}th work -------------------------")
# df.to_csv("고전소설.csv", index=False, encoding="utf-8")
df 

# 일부만 뽑기 (여러 권으로 이루어진 책)

In [7]:
nums = [6, 11, 51, 68, 84, 92, 128]
nums.index(6)

0

In [15]:
from tqdm import tqdm

base_path = "http://www.davincimap.co.kr/davBase/Source/davSourList.jsp?job=C&tab=R&ClssID=DIRC000411&SelID=DIRC000413&worder=TITLE&work=TC"

s = time.time()

df = pd.DataFrame({"path": [],
                   "title": [],
                   "author": [],
                   "page_number": [],
                   "novel_pages": [],
                   "context": []})

# Lang=한글&Page=2

driver = webdriver.Chrome(options=options)
driver.get(base_path)

WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "body")))      # 페이지 기다리기

nums = [6, 11, 51, 68, 84, 92, 128]
novel_ps = [2, 2, 3, 2, 2, 3, 3]


num = 137
for i in range(num):
    print(f"------------------------- {i+1}th / {num}th work start -------------------------")
    print()
    if i in nums:

        novel_pages = novel_ps[nums.index(i)]
            
        results = []
        
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.TAG_NAME, "body")))

        header = driver.find_element(By.XPATH, f'/html/body/div[2]/div/div[2]/table[6]/tbody/tr[{3*i+2}]')
        
        tds = header.find_elements(By.CSS_SELECTOR, "td")

        meta_data = tds[2].find_element(By.CSS_SELECTOR, "div").find_element(By.CSS_SELECTOR, "a")

        path = meta_data.get_attribute("href")    # URL
        title = ""      # 제목
        author = ""     # 작가
        for m in meta_data.find_elements(By.CSS_SELECTOR, "span"):
            title += m.text
        try:
            author = tds[3].find_element(By.CSS_SELECTOR, "div").find_element(By.CSS_SELECTOR, "font").text      # 작가
        except NoSuchElementException as author_exception:
            author = tds[3].find_element(By.CSS_SELECTOR, "div").find_element(By.CSS_SELECTOR, "a").text      # 작가
        

        for j in range(2, novel_pages + 1):
            novel_page = j

            sub_path = path + f"&Lang=%ED%95%9C%EA%B8%80&Page={j}"
            driver.get(sub_path)

            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.TAG_NAME, "body")))
            
            paragraphs = driver.find_elements(By.XPATH, r'/html/body/table[1]/tbody/tr/td[1]/table[3]/tbody/tr[1]/td/table/tbody/tr/td/div')

            page_number_first = 0
            is_seq = False
            body_text = ""  # 본문

            for parag in tqdm(paragraphs):
                p = parag.find_elements(By.CSS_SELECTOR, "div")
                
                try:
                    if not is_seq:
                        page_number_first = p[0].text     # page number
                    pt = p[1].find_elements(By.CSS_SELECTOR, "span")
                    font_style = get_font_style(pt[0])
                    
                    if not body_text:
                        if font_style != "italic":
                            is_seq = False
                            for t in pt:
                                body_text += t.text
                            pair = [path, title, author, page_number_first, novel_page, body_text]
                            results.append(pair)
                            body_text = ""
                        else:
                            is_seq = True
                            for t in pt:
                                body_text += t.text + "\n"
                    else:
                        if font_style != "italic":
                            is_seq = False
                            pair = [path, title, author, page_number_first, novel_page, body_text]
                            results.append(pair)

                            page_number_first = p[0].text
                            body_text = ""
                            for t in pt:
                                body_text += t.text
                            pair = [path, title, author, page_number_first, novel_page, body_text]
                            results.append(pair)

                            body_text = ""
                        else:
                            is_seq = True
                            for t in pt:
                                body_text += t.text        
                except IndexError as e1:
                    pass
                except NoSuchElementException as e2:
                    pass
            driver.back()   # 뒤로 가기

            df_add = pd.DataFrame(results,
                                columns=["path", "title", "author", "page_number", "novel_pages", "context"])
            # 기존 데이터프레임과 병합

            df = pd.concat([df, df_add], axis=0, ignore_index=True)
    e = time.time()

    print(f"Total Elapsed time: {e-s:.2f}s")
    print()
    print(f"------------------------- Done {i+1}th work -------------------------")
# df.to_csv("고전소설.csv", index=False, encoding="utf-8")
df

------------------------- 1th / 137th work start -------------------------

Total Elapsed time: 5.89s

------------------------- Done 1th work -------------------------
------------------------- 2th / 137th work start -------------------------

Total Elapsed time: 5.89s

------------------------- Done 2th work -------------------------
------------------------- 3th / 137th work start -------------------------

Total Elapsed time: 5.89s

------------------------- Done 3th work -------------------------
------------------------- 4th / 137th work start -------------------------

Total Elapsed time: 5.89s

------------------------- Done 4th work -------------------------
------------------------- 5th / 137th work start -------------------------

Total Elapsed time: 5.89s

------------------------- Done 5th work -------------------------
------------------------- 6th / 137th work start -------------------------

Total Elapsed time: 5.89s

------------------------- Done 6th work ------------

100%|██████████| 1581/1581 [01:03<00:00, 24.73it/s]


Total Elapsed time: 75.47s

------------------------- Done 7th work -------------------------
------------------------- 8th / 137th work start -------------------------

Total Elapsed time: 75.47s

------------------------- Done 8th work -------------------------
------------------------- 9th / 137th work start -------------------------

Total Elapsed time: 75.47s

------------------------- Done 9th work -------------------------
------------------------- 10th / 137th work start -------------------------

Total Elapsed time: 75.47s

------------------------- Done 10th work -------------------------
------------------------- 11th / 137th work start -------------------------

Total Elapsed time: 75.47s

------------------------- Done 11th work -------------------------
------------------------- 12th / 137th work start -------------------------



100%|██████████| 337/337 [00:13<00:00, 24.31it/s]


Total Elapsed time: 90.25s

------------------------- Done 12th work -------------------------
------------------------- 13th / 137th work start -------------------------

Total Elapsed time: 90.25s

------------------------- Done 13th work -------------------------
------------------------- 14th / 137th work start -------------------------

Total Elapsed time: 90.25s

------------------------- Done 14th work -------------------------
------------------------- 15th / 137th work start -------------------------

Total Elapsed time: 90.25s

------------------------- Done 15th work -------------------------
------------------------- 16th / 137th work start -------------------------

Total Elapsed time: 90.25s

------------------------- Done 16th work -------------------------
------------------------- 17th / 137th work start -------------------------

Total Elapsed time: 90.25s

------------------------- Done 17th work -------------------------
------------------------- 18th / 137th work s

...

100%|██████████| 1181/1181 [00:45<00:00, 25.71it/s]


Total Elapsed time: 341.13s

------------------------- Done 93th work -------------------------
------------------------- 94th / 137th work start -------------------------

Total Elapsed time: 341.13s

------------------------- Done 94th work -------------------------
------------------------- 95th / 137th work start -------------------------

Total Elapsed time: 341.13s

------------------------- Done 95th work -------------------------
------------------------- 96th / 137th work start -------------------------

Total Elapsed time: 341.13s

------------------------- Done 96th work -------------------------
------------------------- 97th / 137th work start -------------------------

Total Elapsed time: 341.13s

------------------------- Done 97th work -------------------------
------------------------- 98th / 137th work start -------------------------

Total Elapsed time: 341.13s

------------------------- Done 98th work -------------------------
------------------------- 99th / 137th 

100%|██████████| 612/612 [00:25<00:00, 23.99it/s]


Total Elapsed time: 394.48s

------------------------- Done 129th work -------------------------
------------------------- 130th / 137th work start -------------------------

Total Elapsed time: 394.48s

------------------------- Done 130th work -------------------------
------------------------- 131th / 137th work start -------------------------

Total Elapsed time: 394.48s

------------------------- Done 131th work -------------------------
------------------------- 132th / 137th work start -------------------------

Total Elapsed time: 394.48s

------------------------- Done 132th work -------------------------
------------------------- 133th / 137th work start -------------------------

Total Elapsed time: 394.48s

------------------------- Done 133th work -------------------------
------------------------- 134th / 137th work start -------------------------

Total Elapsed time: 394.48s

------------------------- Done 134th work -------------------------
------------------------- 13

,path,title,author,page_number,novel_pages,context
0,http://www.davincimap.co.kr/davBase/Source/dav...,구운몽(九雲夢),김만중,1,2.0,구운몽 하
1,http://www.davincimap.co.kr/davBase/Source/dav...,구운몽(九雲夢),김만중,2,2.0,"천자가 환자(宦者)를 보내어 상서를 부르자, 환자가 정사도의 집에 가 물으니 상서가..."
2,http://www.davincimap.co.kr/davBase/Source/dav...,구운몽(九雲夢),김만중,3,2.0,「내 이태백을 보지 못하여 한이었는데 경을 얻었으니 어찌 이태백을 부러워하겠는가? ...
3,http://www.davincimap.co.kr/davBase/Source/dav...,구운몽(九雲夢),김만중,4,2.0,"하고, 즉시 궁녀를 명하여 백옥으로 된 책상과 유리 벼루와 금으로 만든 두꺼비 모양..."
4,http://www.davincimap.co.kr/davBase/Source/dav...,구운몽(九雲夢),김만중,5,2.0,황제가 말하였다.
...,...,...,...,...,...,...
5526,http://www.davincimap.co.kr/davBase/Source/dav...,현수문전 (경판본)(玄壽文傳),,293,3.0,"왕이 이 말을 듣고 희허(喜許) 탄하며 이르기를,"
5527,http://www.davincimap.co.kr/davBase/Source/dav...,현수문전 (경판본)(玄壽文傳),,294,3.0,"「선왕이 매양 석참정 산소에 자주 친행(親行)하심을 과인이 잊지 아니하였으나, 그 ..."
5528,http://www.davincimap.co.kr/davBase/Source/dav...,현수문전 (경판본)(玄壽文傳),,295,3.0,하고 즉시 발생하여 석참정 산소의 가 정성으로 제하고 돌아와 정사를 다스리니 위국이...
5529,http://www.davincimap.co.kr/davBase/Source/dav...,현수문전 (경판본)(玄壽文傳),,296,3.0,"세월이 여류하여 위왕의 나이 사십이 되매 삼자 일녀를 두고, 여러 형제 다 각각 자..."


In [25]:
import numpy as np

page_nums = np.array(df.loc[df["title"] == '구운몽(九雲夢)', "page_number"])

page_nums = page_nums.astype(int)

# 연속하지 않는 자연수를 저장할 리스트
non_continuous_nums = []

# 연속하지 않는 자연수가 있는지 확인하고 추가
for i in range(1, len(page_nums)):
    if page_nums[i] != page_nums[i-1] + 1:
        non_continuous_nums.append(page_nums[i])

non_continuous_nums

[53]

In [ ]:
max_pages = [734, 178, [372, 500], 482, 344, [317, 357], [338, 293]]
titles = ['구운몽(九雲夢)', '금방울전(金鈴傳)', '숙향전(淑香傳)', '열녀춘향수절가(烈女春香守節歌)', '유충렬전(劉忠烈傳)', '임진록(壬辰錄)', '현수문전 (경판본)(玄壽文傳)']

In [37]:
df2 = df.copy()

In [40]:
338+293

631

In [43]:
df2["page_number"] = df2["page_number"].astype(int)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531 entries, 0 to 5530
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   path         5531 non-null   object 
 1   title        5531 non-null   object 
 2   author       5531 non-null   object 
 3   page_number  5531 non-null   int32  
 4   novel_pages  5531 non-null   float64
 5   context      5531 non-null   object 
dtypes: float64(1), int32(1), object(4)
memory usage: 237.8+ KB


In [45]:
df2.loc[(df2['title'] == '구운몽(九雲夢)') & (df2['novel_pages'] == 2.0), 'page_number'] += 734

df2.loc[(df2['title'] == '금방울전(金鈴傳)') & (df2['novel_pages'] == 2.0), 'page_number'] += 178

df2.loc[(df2['title'] == '숙향전(淑香傳)') & (df2['novel_pages'] == 2.0), 'page_number'] += 372
df2.loc[(df2['title'] == '숙향전(淑香傳)') & (df2['novel_pages'] == 3.0), 'page_number'] += 872

df2.loc[(df2['title'] == '열녀춘향수절가(烈女春香守節歌)') & (df2['novel_pages'] == 2.0), 'page_number'] += 482

df2.loc[(df2['title'] == '유충렬전(劉忠烈傳)') & (df2['novel_pages'] == 2.0), 'page_number'] += 344

df2.loc[(df2['title'] == '임진록(壬辰錄)') & (df2['novel_pages'] == 2.0), 'page_number'] += 317
df2.loc[(df2['title'] == '임진록(壬辰錄)') & (df2['novel_pages'] == 3.0), 'page_number'] += 674

df2.loc[(df2['title'] == '현수문전 (경판본)(玄壽文傳)') & (df2['novel_pages'] == 2.0), 'page_number'] += 338
df2.loc[(df2['title'] == '현수문전 (경판본)(玄壽文傳)') & (df2['novel_pages'] == 3.0), 'page_number'] += 631

In [47]:
df2.loc[df2['title'] == '구운몽(九雲夢)', "page_number"]

0       735
1       736
2       737
3       738
4       739
       ... 
780    1518
781    1519
782    1520
783    1521
784    1522
Name: page_number, Length: 785, dtype: int32

In [ ]:
817 - 

In [48]:
df2.to_csv("고전소설(추가).csv", index=False)

In [1]:
import pandas as pd

df = pd.read_csv("고전소설(추가).csv")

,path,title,author,page_number,novel_pages,context
0,http://www.davincimap.co.kr/davBase/Source/dav...,구운몽(九雲夢),김만중,735,2.0,구운몽 하
1,http://www.davincimap.co.kr/davBase/Source/dav...,구운몽(九雲夢),김만중,736,2.0,"천자가 환자(宦者)를 보내어 상서를 부르자, 환자가 정사도의 집에 가 물으니 상서가..."
2,http://www.davincimap.co.kr/davBase/Source/dav...,구운몽(九雲夢),김만중,737,2.0,「내 이태백을 보지 못하여 한이었는데 경을 얻었으니 어찌 이태백을 부러워하겠는가? ...
3,http://www.davincimap.co.kr/davBase/Source/dav...,구운몽(九雲夢),김만중,738,2.0,"하고, 즉시 궁녀를 명하여 백옥으로 된 책상과 유리 벼루와 금으로 만든 두꺼비 모양..."
4,http://www.davincimap.co.kr/davBase/Source/dav...,구운몽(九雲夢),김만중,739,2.0,황제가 말하였다.
...,...,...,...,...,...,...
5229,http://www.davincimap.co.kr/davBase/Source/dav...,현수문전 (경판본)(玄壽文傳),NaN,627,2.0,"하며 칼을 빼어 스스로 멱을 찔러 죽으니, 모든 군사가 손을 묶어 살기를 빌거늘, ..."
5230,http://www.davincimap.co.kr/davBase/Source/dav...,현수문전 (경판본)(玄壽文傳),NaN,628,2.0,"크게 괴이 여겨 인하여 죽이고 황성의 들어와 적진을 함몰(陷沒)함을 주달하오니, 상..."
5231,http://www.davincimap.co.kr/davBase/Source/dav...,현수문전 (경판본)(玄壽文傳),NaN,629,2.0,"「먼저 경이 이르러 오매 짐이 보고 마음을 놓이더니, 이제 승전함을 들으니 마음이 ..."
5232,http://www.davincimap.co.kr/davBase/Source/dav...,현수문전 (경판본)(玄壽文傳),NaN,630,2.0,"하시고 제장 군졸을 상사하시며 만조를 모아 크게 잔치하시고, 사방에 방 부처 백성을..."


In [4]:
df_dup = df[df["title"].isin(["숙향전(淑香傳)", "임진록(壬辰錄)", "현수문전 (경판본)(玄壽文傳)"])]

df_dup[df_dup["context"].duplicated()]

,path,title,author,page_number,novel_pages,context
970,http://www.davincimap.co.kr/davBase/Source/dav...,숙향전(淑香傳),미상,390,2.0,"하니 이랑이 말하기를,"
974,http://www.davincimap.co.kr/davBase/Source/dav...,숙향전(淑香傳),미상,394,2.0,"하니 이랑이 말하기를,"
1000,http://www.davincimap.co.kr/davBase/Source/dav...,숙향전(淑香傳),미상,420,2.0,"하니 할미가 말하기를,"
1006,http://www.davincimap.co.kr/davBase/Source/dav...,숙향전(淑香傳),미상,426,2.0,"하니 이랑이 말하기를,"
1010,http://www.davincimap.co.kr/davBase/Source/dav...,숙향전(淑香傳),미상,430,2.0,"이랑이 말하기를,"
...,...,...,...,...,...,...
5286,http://www.davincimap.co.kr/davBase/Source/dav...,현수문전 (경판본)(玄壽文傳),NaN,684,3.0,"위왕 이르기를,"
5296,http://www.davincimap.co.kr/davBase/Source/dav...,현수문전 (경판본)(玄壽文傳),NaN,694,3.0,하더라.
5411,http://www.davincimap.co.kr/davBase/Source/dav...,현수문전 (경판본)(玄壽文傳),NaN,809,3.0,하였더라.
5419,http://www.davincimap.co.kr/davBase/Source/dav...,현수문전 (경판본)(玄壽文傳),NaN,817,3.0,하였더라.


In [7]:
df_dup.loc[970, "path"]

'http://www.davincimap.co.kr/davBase/Source/davSource.jsp?Job=Body&SourID=SOUR001728'